# **Setup the development framework such that changes in py files auto load into this file**
- You can ignore this if your interest is only in the business logic and design. 
- The steps in this section are only environment set related

### **Make sure py file change are loaded automatically**

In [ ]:
%load_ext autoreload
%autoreload 2

### **Import basic package required for this assignment**

In [ ]:
import os
import pandas as pd
import db.constants as C
import numpy as np


### **Make sure right sources are getting picked**

In [ ]:
from dao.base_transformer import BaseDBTransformer

### **Make sure the latest class are getting loaded.**

In [ ]:
# 1. Import the modules first (so Python knows them)
import importlib
import service.product_service
import service.cart_service
import service.purchase_service
import service.discount_service
import dao.base_transformer
import dao.cart_transformer
import dao.discount_transformer
import dao.product_transformer
import dao.purchase_transformer
import db.constants
import db.dbsql

# 2. Reload them (useful after edits)
importlib.reload(service.product_service)
importlib.reload(service.cart_service)
importlib.reload(service.purchase_service)
importlib.reload(service.discount_service)
importlib.reload(db.constants)
importlib.reload(dao.base_transformer)
importlib.reload(dao.product_transformer)
importlib.reload(dao.cart_transformer)
importlib.reload(dao.discount_transformer)
importlib.reload(dao.purchase_transformer)

# 3. Import the classes fresh from the reloaded modules
from service.product_service import ProductService
from service.cart_service import CartService
from service.purchase_service import PurchaseService
from service.discount_service import DiscountService
from dao.product_transformer import ProductTransformer
from dao.discount_transformer import DiscountTransformer
from dao.cart_transformer import CartTransformer
from dao.purchase_transformer import PurchaseTransformer
from dao.base_transformer import BaseDBTransformer

import db.constants
import random
from db.dbsql import SessionLocal, Product, ProductClass

# **Transformers**
- This section is the core design of the project
- There are four transformers 
   - Products transformer: This is the database of the list of the products in EStore. The prodcuts.csv file for demo has 9 products. 
   - Cart transformer: This is user selected products added into the cart instance specific to the user. 
   - purchase transfromer: This is checkout functionaility of the cart. 
        - It has two kinds of purchases. With discount by applying discount code.
        - It also supports purchase without applying discount code. If no discount is applied it generates a ne discount code and adds to discounts for the subsequent purchases. 
   - Discount transformer: This handles the discounts codes after the purchase. 

   - For each these transforers there is .py  file and .csv file in the src directory. 
   - Each transfomer derives from the base transfomer class which manages the csv read/write for that transformer. 
   - The base transformer class also handles basis CRUD operation if they are at the level of the dataframe(entire row level operations) without having to deal with individual columns (as columns are specific to the transformer)

- The code is organized as a typical pyproject.toml file. 

## **Product Transformer - demonstration**

#### **Get the product**

In [ ]:
pt = ProductTransformer()
prds_all = pt.list_all_products()
prds_all.head(5)

In [ ]:
prds_all.to_dict(orient="records")

In [ ]:
prds_2050 = pt.list_products_in_class_df(2050)
prds_2050.to_dict(orient='records')

In [ ]:
rows = BaseDBTransformer.read(C.prdc)
rows


#### **Get a single product**

In [ ]:
df = pt.list_all_products(15)
print(type(df))
df

In [ ]:
rows = BaseDBTransformer.read(C.prd, 25)
dict = rows.to_dict(orient="records")[0]
dict

#### **Get the product classes and description**

In [ ]:
prod_classes = pt.get_products_class()

for i,pclass in enumerate(prod_classes[C.ptyp]):
    print("Iteration i ", i, " p class code = ", pclass)
    if ( i>3 ):
        break
    pc_dict = pt.list_products_in_class(pclass)
    print(pc_dict)

prod_classes


## **Discount Transformer - demonstration**

In [ ]:
dt = DiscountTransformer()
dis_all = dt.list_all_codes()
dis_all

## **Purchase Transformer - demonstration**

In [ ]:
prt = PurchaseTransformer()
prt.list_orders_by_customer(3)

## **Cart Transformer - demonstration**

In [ ]:
ct = CartTransformer
ct.list_all()

# **Business logic - Helper functions**

### **Get a random selection of 5 products to purchase**

In [ ]:
products_to_buy = pt.getRandomProducts(5)
customer_id=3


In [ ]:
products_to_buy

In [ ]:
BaseDBTransformer.read(C.prd, 58)

### **Add these products to the cart of a selected customer**

##### **Select the items**

In [ ]:
selected_items = products_to_buy.loc[:, [C.pid]]
selected_items[C.qnt] = np.random.randint(1, 3, size=len(selected_items))
items = selected_items.to_dict(orient='records')
items


##### **Add Them to the cart**

In [ ]:
CartTransformer.addToCart(customer_id, items, debug=True)

##### **Verify the addition**

In [ ]:
print("customer_id = ", customer_id)
# CartTransformer.list_cust(customer_id)
# BaseDBTransformer.read(C.cart, **{C.custid:customer_id})
# BaseDBTransformer.readf(C.cart, **{C.custid+"__eq":customer_id})
BaseDBTransformer.read(C.cart, customer_id)

#### **Verify empty cart**

In [ ]:
# PurchaseTransformer.empty_cart(3)

### **Purchase the items in the cart**

In [ ]:
order_id = PurchaseService.purchase_cart_items(customer_id, debug=True)
print("Order Id =", order_id)


##### **Delete a purchase**

In [ ]:
PurchaseService.deletePurchase(order_id, customer_id)

### **Purchase cart items with discount coupon**

In [ ]:
customer_id = 3
order_id = "10079"
discount_id = DiscountTransformer.generate_discount_id(order_id, customer_id)
print(customer_id, order_id, discount_id)

In [ ]:
cust_cart = BaseDBTransformer.read(C.cart, **{C.custid: customer_id})
for i, dict in enumerate(cust_cart.to_dict(orient='records')):
    prd_df = BaseDBTransformer.read(C.prd, dict[C.pid])
    prd_dict = prd_df.iloc[0].to_dict()
    print(prd_dict)
print(cust_cart)


In [ ]:
print(discount_id)
res_with_d = PurchaseService.purchase_discounted_cart_items(customer_id, discount_id, debug=True)
print( " Order Id with discount =", res_with_d)


In [ ]:
discount_id = "3__10079"
# BaseDBTransformer.read(C.discounts)
BaseDBTransformer.read(C.discounts, discount_id)

In [ ]:
DiscountService.list_eligible_codes(3)

# **Admin APIs**

### **Get all discount codes**

In [ ]:
all_disc_codes = dt.list_all_codes()
all_disc_codes


### **Get codes eligible - But not enabled**

In [ ]:
eligible_disc_codes = dt.list_eligible_codes(customer_id)
eligible_disc_codes

### **Get all active codes - But not yet used by the customer**

In [ ]:
active_disc_codes = dt.list_active_codes()
active_disc_codes

#### **Get used discount codes by the customer**

In [ ]:
dt.list_used_codes(customer_id)

### **Enable a discount code for a customer**

#### **First change the status**

In [ ]:
# discount id is custoemrid + __ + Order_id
discount_id = "3__10076" 
dt.enable_discount_codes(discount_id, 10, "NEW_CUSTOMER")

#### **Second verify the record that status has really changed**

In [ ]:
active_disc_codes = dt.list_active_codes()
active_disc_codes

#### **Check Orders and items by customer**

In [ ]:
# print(order_id)
print( PurchaseTransformer.list_orders_by_customer(customer_id) )
df = BaseDBTransformer.read(C.items, "10077")
df


### **Test the FASTAPIs**

In [ ]:
df = ProductTransformer.get_products_class()
mapped = df.to_dict(orient="records")
mapped

### **Invoices**

In [ ]:
def generate_invoice(cust_id: int):
            orders = BaseDBTransformer.read(C.orders, **{C.custid:cust_id})
            if(orders.empty):
                print("Nor order from the customer_id = ", cust_id, " Nothing to geenrate. Aborting. ")
                return
            orditems = BaseDBTransformer.readdf(C.items, C.ordid, orders[C.ordid].to_list())
            if(orders.empty):
                print("Nor order from the customer_id = ", cust_id, " Nothing to geenrate. Aborting. ")
                return
            products = BaseDBTransformer.readdf(C.prd, C.pid, orditems[C.pid].unique().tolist())
            if(products.empty):
                print( "No Valid product Ids. Nothing to generate for customer = ", cust_id)
                return
            discs = BaseDBTransformer.readdf(C.discounts, C.did, orders[C.did].unique().tolist())
            discs = discs.loc[discs[C.dpct] > 0, [C.did, C.dpct, C.dst]]
            customer = BaseDBTransformer.readdf(C.custs, C.custid, [cust_id])
            if(customer.empty):
                print("Custoemr does not exist. Aborting.")
                return;

            customer_invoice = orders.merge(orditems, on=C.ordid).merge(products, on=C.pid).merge(discs, on=C.did, how='left')
            customer_invoice["FINAL COST"] = (
            customer_invoice[C.mrp].astype(float) *
            customer_invoice[C.qnt] *
            (100 - customer_invoice[C.dpct].fillna(0)) / 100
            )
            customer_invoice["ORIGINAL COST"] = (
            customer_invoice[C.mrp].astype(float) *
            customer_invoice[C.qnt]
            )

            cigrp_p = (
                    customer_invoice.groupby([C.ordid, C.pid]).agg(
                            order_number=(C.ordid, 'max'),
                            Product_Name=(C.pname, 'max'),
                            discount_code=(C.did, 'max'),
                            discount_percentage=(C.dpct, 'max'),
                            original_cost=('ORIGINAL COST', 'sum'),
                            final_cost=('FINAL COST', 'sum')
                    ).reset_index(level=1, drop=True)
            )
            cigrp_p['order'] =1

            cigrp_o = (
                    customer_invoice.groupby([C.ordid]).agg(
                            order_number=(C.ordid, 'max'),
                            Product_Name=(C.pname, lambda x: 'SUB TOTAL'),
                            discount_code=(C.did, 'max'),
                            discount_percentage=(C.dpct, 'max'),
                            original_cost=('ORIGINAL COST', 'sum'),
                            final_cost=('FINAL COST', 'sum')
                    )
            )
            cigrp_o['order'] =0

            cigrp_o.index = [f"SUMMARY-{idx}" for idx in cigrp_o.index]

            cigrp_g = pd.DataFrame({
                    "order_number": f"{customer.loc[0, C.custfn]} {customer.loc[0, C.custln]}",
                    "Product_Name" : f"GRAND TOTAL - {customer.loc[0, C.custfn]} {customer.loc[0, C.custln]}",
                    "discount_code"  : ",".join(customer_invoice[C.did].unique()),
                    "discount_percentage" :f"Discount range {customer_invoice[C.dpct].min()} - {customer_invoice[C.dpct].max()}",
                    "original_cost" : customer_invoice['ORIGINAL COST'].sum(), 
                    "final_cost" : customer_invoice['FINAL COST'].sum()
            }, index=['OVERALL SUMMARY'])
            cigrp_g['order'] = 2

            invoice = pd.concat([
                    cigrp_p.fillna(0),
                    cigrp_o.fillna(0),
                    cigrp_g.fillna(0)
            ])
            invoice.index.name = "Order Numbers"
            retdf = invoice.sort_values(by=['order_number', 'order', 'original_cost'], ascending=[True, True, True]).loc[:,['Product_Name','discount_code', "discount_percentage", 'original_cost', 'final_cost']]
            # retdf.index.name = "Sno"
            #print( f"Invoice for Customer {customer.loc[0, C.custfn]} {customer.loc[0, C.custln]}:\n\nTotal = {customer_invoice['FINAL COST'].sum()}\n")
            return  retdf

In [ ]:
cust_id=4
orders = BaseDBTransformer.read(C.orders, **{C.custid:cust_id})
orditems = BaseDBTransformer.readdf(C.items, C.ordid, orders[C.ordid].to_list())
products = BaseDBTransformer.readdf(C.prd, C.pid, orditems[C.pid].unique().tolist())
discs = BaseDBTransformer.readdf(C.discounts, C.did, orders[C.did].unique().tolist())
discs = discs.loc[discs[C.dpct] > 0, [C.did, C.dpct, C.dst]]
customer = BaseDBTransformer.readdf(C.custs, C.custid, [cust_id])

print(orders)
print(orditems)
print(orditems[C.pid].unique().tolist())
df = BaseDBTransformer.read(C.prd, **{C.pid:229})
df

In [ ]:
prds_all[C.pid].unique()
orders = BaseDBTransformer.read(C.orders)
items =  BaseDBTransformer.read(C.items)
items[C.pid].unique()
custs = BaseDBTransformer.read(C.custs)

In [ ]:
prds_all.empty


In [ ]:
generate_invoice(10)

